In [1]:
import pandas as pd
import sys
import logging
import json
from llm_api_client import LLMAPIClient

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Only log warnings and errors to avoid cluttering the output
logging.getLogger().setLevel(logging.WARNING)

logger = logging.getLogger(__name__)

In [3]:
prompt = """다음은 학생이 제출한 수행평가 과제물이다. 제시된 평가 기준을 참고하여 과제물을 체계적으로 분석하고 평가하라. 평가 시 다음 단계를 순차적으로 따르라:

1. 평가기준을 꼼꼼히 읽고 이해하라.
2. 학생의 제출물을 세심히 분석하며, 평가 기준에 부합하거나 미흡한 점이 있는지 확인하라. 만약 직접적으로 인용할 수 있는 표현이 있다면, 해당 표현을 정확히 인용하라.
3. 인용한 표현이 있다면 평가 기준과의 관계를 논리적으로 설명하라.
4. 평가 항목별로 정량적 점수를 부여하며 근거를 구체적으로 기술하라.

학생 제출물:
{학생_제출물}"""

system_instruction ="""너는 고등학교 {과목} 담당 교사이며, 학생이 제출한 {수행평가_제목} 수행평가 과제물을 객관적이고 공정하게 평가하는 전문 평가자 역할을 수행한다. 다음 지침을 명확히 숙지하여 평가하라.
수행평가에 대한 세부정보는 다음과 같다:

학년: {학년}
수행평가 제목: {수행평가_제목}
수행평가 유형: {수행평가_유형}
수행평가 설명: {수행평가_설명}
평가 기준:
{평가_기준}

평가 수행 지침
1. 평가 기준 엄격 준수
- 제공된 평가 기준 및 체크리스트의 항목을 정확히 준수하여 평가하라. 반드시 모든 항목 및 세부항목에 대해 평가하고, 누락된 항목이 없도록 하라.

2. 직접적 인용 및 명확한 설명
- 학생의 제출물에서 인용된 표현을 근거로 사용한다면 해당 표현이 평가 기준에 적합하거나 미흡한 이유를 구체적으로 설명하라.

3. 논리적이고 객관적인 근거
- 평가 결과가 일관되도록 하고, 점수 부여의 근거를 명확히 기술하여 객관성과 신뢰성을 높이도록 하라.

결과는 JSON 객체로만 출력하라. 마크다운 코드 펜스(```)는 포함하지 말고, 인용된 표현을 위해 문자열 내부에 큰따옴표(")를 포함해야 할 경우 반드시 \"처럼 이스케이프 처리하라"""

output_schema = {
    "type": "object",
    "properties": {
        "evaluation": {
            "type": "array",
            "description": "평가 항목별 점수와 이유",
            "items": {
                "type": "object",
                "properties": {
                    "title": {
                        "type": "string",
                        "description": "평가 대분류"
                    },
                    "subcategories": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "name": {
                                    "type": "string",
                                    "description": "평가 소분류"
                                },
                                "score": {
                                    "type": "integer",
                                    "description": "0~3 사이의 점수"
                                },
                                "reason": {
                                    "type": "string",
                                    "description": "점수 평가 이유"
                                },
                                "evidence":{
                                    "type": "array",
                                    "description": "관련 있는 텍스트를 증거로 제시",
                                    "items": {
                                        "type": "string",
                                        "description": "증거 텍스트"
                                    }
                                }
                            },
                            "required": ["name", "score", "reason", "evidence"]
                        }
                    }
                },
                "required": ["title", "subcategories"]
            }
        },
        "overall_feedback": {
            "type": "string",
            "description": "전체적인 피드백"
        }
    },
    "required": ["evaluation", "overall_feedback"]
}

In [4]:
df_aihub = pd.read_csv('C:/Users/yuhan/Desktop/CSD_18기/베어러블/AI_Hub/에세이_글_평가_데이터_전처리완.csv')

good_essay_ids = ["ESSAY_71375", "ESSAY_65900", "ESSAY_43915"]
bad_essay_ids = ["ESSAY_78467", "ESSAY_76539", "ESSAY_76550"]

essay_dict = {}
for essay_id in good_essay_ids + bad_essay_ids:
    essay_dict[essay_id] = df_aihub.loc[df_aihub['essay_id'] == essay_id, :].to_dict(orient='records')[0]

In [5]:
df_aihub.head()

,student_grade,essay_id,essay_type,essay_main_subject,essay_prompt,processed_text,essay_score_org,essay_score_cont,essay_score_exp
0,고등_1학년,ESSAY_33982,글짓기,소중함을 잃었던 경험,익숙함에 속아 소중함을 잊지 말자는 명언 (Do not get fooled by ...,익숙함에 속아 소중함을 잊지말자라는 명언은 매우 유명한 명언이다. 난 이 명언의 내...,"[[3, 2, 3, 3], [2, 2, 3, 3], [2, 2, 2, 3]]","[[3, 2, 0, 3], [3, 2, 0, 2], [3, 2, 0, 2]]","[[3, 3, 2], [2, 2, 2], [3, 3, 2]]"
1,고등_1학년,ESSAY_33985,글짓기,소중함을 잃었던 경험,익숙함에 속아 소중함을 잊지 말자는 명언 (Do not get fooled by ...,소중함이란 가치를 따질 수 없을 만큼 매우 귀중한 것을 말합니다. 저에게도 이러한 ...,"[[2, 3, 3, 3], [3, 3, 3, 3], [2, 2, 3, 3]]","[[3, 2, 0, 3], [3, 3, 0, 3], [3, 3, 0, 3]]","[[3, 3, 2], [2, 2, 2], [2, 3, 2]]"
2,고등_1학년,ESSAY_33986,글짓기,소중함을 잃었던 경험,익숙함에 속아 소중함을 잊지 말자는 명언 (Do not get fooled by ...,"""익숙함에 속아 소중함을 잊지 말자"" 모든 사람들이 아는 말이다. 그만큼 모두가 그...","[[2, 3, 3, 2], [3, 3, 3, 2], [2, 2, 2, 3]]","[[3, 2, 0, 3], [3, 2, 0, 3], [3, 2, 0, 2]]","[[3, 3, 2], [3, 3, 3], [2, 2, 2]]"
3,고등_1학년,ESSAY_33988,글짓기,소중함을 잃었던 경험,익숙함에 속아 소중함을 잊지 말자는 명언 (Do not get fooled by ...,나는 가까운 사람에 대한 소중함에 대하여 알지 못했다. 그 일이 있기 전까지 말이다...,"[[3, 3, 3, 3], [2, 2, 3, 3], [2, 2, 2, 3]]","[[3, 3, 0, 3], [3, 3, 0, 2], [2, 2, 0, 2]]","[[3, 3, 3], [3, 3, 2], [2, 3, 2]]"
4,고등_1학년,ESSAY_33989,글짓기,소중함을 잃었던 경험,익숙함에 속아 소중함을 잊지 말자는 명언 (Do not get fooled by ...,나는 태어났을 때부터 할머니와 함께 살아왔는데 할머니는 하루도 빠짐없이 나에게 아침...,"[[2, 2, 3, 3], [2, 3, 3, 2], [3, 3, 3, 3]]","[[3, 2, 0, 3], [3, 3, 0, 2], [3, 2, 0, 3]]","[[3, 3, 3], [3, 3, 3], [3, 3, 3]]"


In [5]:
with open('checklists/논술형_checklist.json', 'r', encoding='utf-8') as f:
    checklist = json.load(f)

def replace_placeholders(template, replacements):
    """Replace placeholders in template with values from replacements dictionary
    
    Args:
        template: The template string with {placeholders}
        replacements: Dictionary with placeholder names as keys and replacement values as values
        
    Returns:
        String with placeholders replaced by actual values
    """
    result = template
    for placeholder, value in replacements.items():
        result = result.replace(f"{{{placeholder}}}", str(value))
    return result

def format_template(template, values, checklist):
    """Format a template with the given values
    
    Args:
        template: The template string with {placeholders}
        values: Dictionary of placeholder values
        checklist: Checklist data structure
        
    Returns:
        Formatted template with placeholders replaced
    """
    replacements = {
        '학년': values.get('student_grade', ''),
        '과목': values.get('subject', '국어'),
        '수행평가_제목': values.get('essay_main_subject', ''),
        '수행평가_유형': values.get('essay_type', ''),
        '수행평가_설명': values.get('essay_prompt', ''),
        '평가_기준': json.dumps(checklist, ensure_ascii=False),
        '학생_제출물': values.get('processed_text', '')
    }
    
    return replace_placeholders(template, replacements)


In [6]:
# Initialize the client
client = LLMAPIClient()
client.logger = logger

In [7]:
models_list = [
    "gemini-2.5-flash",
    "gemini-2.5-flash-lite-preview-06-17",
    "gemini-2.0-flash",
    "gemini-2.0-flash-lite",
    "claude-sonnet-4-20250514",
    "claude-3-7-sonnet-20250219",
    "claude-3-5-haiku-20241022",
    "claude-3-5-sonnet-20241022",
    "gpt-4.1",
    "gpt-4.1-mini",
    "gpt-4o",
    "gpt-4o-mini",
    "o4-mini",
    "o3-mini"
]

light_models_list = ['gemini-2.0-flash-lite', 'claude-3-5-haiku-20241022', 'gpt-4o-mini']

In [8]:
import time

all_processed_results = []

for essay_id, essay_data in essay_dict.items():
    for trial in range(1, 4):
        formatted_prompt = format_template(prompt, essay_data, checklist)
        formatted_system_instruction = format_template(system_instruction, essay_data, checklist)

        params = {
            "prompt": formatted_prompt,
            "system_instruction": formatted_system_instruction,
            "models": models_list,
            "response_schema": output_schema,
            "file_path": None,
            "temperature": 0.1,
            "max_tokens": 8000,
            "enable_thinking": False
        }

        essay_results = await client.generate_responses(**params)

        client.save_results(essay_results, file_path=f"auto_evaluation_results/evaluation_results_json_set2/{essay_id}_evaluation_results_trial_{trial}.json")

        try:
            processed_results = client.process_results(essay_results, essay_id=essay_id, trial=trial)
            all_processed_results.extend(processed_results)
        except Exception as e:
            logger.error(f"Error processing results for essay {essay_id}, trial {trial}: {e}")
            continue

        # Sleep to avoid rate limiting
        time.sleep(30)

2025-06-25 12:19:50,439 - WARNING - gpt-4.1-mini의 응답이 유효한 JSON이 아닙니다. 원시 텍스트를 반환합니다.
2025-06-25 12:22:46,681 - WARNING - gpt-4.1-mini의 응답이 유효한 JSON이 아닙니다. 원시 텍스트를 반환합니다.
2025-06-25 12:25:30,843 - WARNING - claude-3-5-sonnet-20241022의 응답이 유효한 JSON이 아닙니다. 원시 텍스트를 반환합니다.
2025-06-25 12:25:31,085 - WARNING - gpt-4.1-mini의 응답이 유효한 JSON이 아닙니다. 원시 텍스트를 반환합니다.
2025-06-25 12:31:59,797 - WARNING - gpt-4.1-mini의 응답이 유효한 JSON이 아닙니다. 원시 텍스트를 반환합니다.
2025-06-25 12:34:50,619 - WARNING - gemini-2.5-flash-lite-preview-06-17의 응답이 유효한 JSON이 아닙니다. 원시 텍스트를 반환합니다.
2025-06-25 12:34:51,064 - WARNING - gpt-4.1-mini의 응답이 유효한 JSON이 아닙니다. 원시 텍스트를 반환합니다.
2025-06-25 12:37:21,612 - WARNING - gemini-2.5-flash-lite-preview-06-17의 응답이 유효한 JSON이 아닙니다. 원시 텍스트를 반환합니다.
2025-06-25 12:37:21,725 - WARNING - claude-sonnet-4-20250514의 응답이 유효한 JSON이 아닙니다. 원시 텍스트를 반환합니다.
2025-06-25 12:37:21,775 - WARNING - claude-3-5-haiku-20241022의 응답이 유효한 JSON이 아닙니다. 원시 텍스트를 반환합니다.
2025-06-25 12:38:59,589 - WARNING - gemini-2.5-flash-lite-preview

In [9]:
pd.DataFrame(all_processed_results)

,metadata,response
0,"{'model': 'gemini-2.5-flash', 'response_time':...","{'text': {'evaluation': [{'title': '표현', 'subc..."
1,{'model': 'gemini-2.5-flash-lite-preview-06-17...,"{'text': {'evaluation': [{'title': '표현', 'subc..."
2,"{'model': 'gemini-2.0-flash', 'response_time':...","{'text': {'evaluation': [{'title': '표현', 'subc..."
3,"{'model': 'gemini-2.0-flash-lite', 'response_t...","{'text': {'evaluation': [{'title': '표현', 'subc..."
4,"{'model': 'claude-sonnet-4-20250514', 'respons...","{'text': {'evaluation': [{'title': '표현', 'subc..."
...,...,...
247,"{'model': 'gpt-4.1-mini', 'response_time': 31....","{'text': {'evaluation': [{'title': '표현', 'subc..."
248,"{'model': 'gpt-4o', 'response_time': 13.226997...","{'text': {'evaluation': [{'title': '표현', 'subc..."
249,"{'model': 'gpt-4o-mini', 'response_time': 18.8...","{'text': {'evaluation': [{'title': '표현', 'subc..."
250,"{'model': 'o4-mini', 'response_time': 19.36007...","{'text': {'evaluation': [{'title': '표현', 'subc..."


In [10]:
client.extract_evaluation_scores(all_processed_results)

,model,response_time,timestamp,essay_id,trial,문법의 정확성,단어 사용의 적절성,문장 표현의 적절성,문단 내 구조의 적절성,문단 간 구조의 적절성,구조의 일관성,분량의 적절성,주제의 명료성,설명의 구체성,사고의 창의성,프롬프트 독해,구성,내용
0,gemini-2.5-flash,12.162923,2025-06-25T12:17:05.006205,ESSAY_71375,1,2,2,2,2.0,1.0,1.0,3.0,1.0,2.0,2.0,2.0,NaN,NaN
1,gemini-2.5-flash-lite-preview-06-17,4.949075,2025-06-25T12:17:05.006205,ESSAY_71375,1,2,2,2,2.0,2.0,2.0,3.0,2.0,2.0,2.0,3.0,NaN,NaN
2,gemini-2.0-flash,8.460104,2025-06-25T12:17:05.006205,ESSAY_71375,1,3,2,2,2.0,2.0,2.0,3.0,3.0,2.0,2.0,3.0,NaN,NaN
3,gemini-2.0-flash-lite,11.648617,2025-06-25T12:17:05.006205,ESSAY_71375,1,2,2,2,2.0,1.0,1.0,3.0,2.0,2.0,2.0,3.0,NaN,NaN
4,claude-sonnet-4-20250514,43.503310,2025-06-25T12:17:36.357039,ESSAY_71375,1,2,2,1,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,gpt-4.1-mini,31.415680,2025-06-25T12:51:11.222766,ESSAY_76550,3,2,2,2,2.0,2.0,2.0,3.0,3.0,2.0,2.0,3.0,NaN,NaN
226,gpt-4o,13.226998,2025-06-25T12:51:11.222766,ESSAY_76550,3,2,2,2,2.0,2.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
227,gpt-4o-mini,18.885490,2025-06-25T12:51:11.222766,ESSAY_76550,3,2,2,2,2.0,2.0,2.0,2.0,3.0,2.0,2.0,3.0,NaN,NaN
228,o4-mini,19.360076,2025-06-25T12:51:11.222766,ESSAY_76550,3,2,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Read log file logs\api_interactions_20250621_150813.log
# Only read in ERROR and WARNING logs
error_logs = []
with open('logs/api_interactions_20250621_150813.log', 'r', encoding='utf-8') as f:
    for line in f:
        if 'ERROR' in line or 'WARNING' in line or 'Results saved to' in line:
            error_logs.append(line.strip())

| Essay ID | Model                              | Trial | Error Type                            |
|----------|-------------------------------------|-------|----------------------------------------|
| 71375    | o3-mini                             | 5     | Incomplete response (token limit)      |
| 65900    | gemini-2.5-flash-lite-preview-06-17 | 1     | Incomplete response (token limit)      |
| 65900    | gemini-2.5-flash-lite-preview-06-17 | 2     | NoneType object (subscript)            |
| 65900    | gpt-4.1-mini                        | 2     | Incomplete response (token limit)      |
| 65900    | gemini-2.5-flash-lite-preview-06-17 | 3     | Incomplete response (token limit)      |
| 65900    | gemini-2.5-flash-lite-preview-06-17 | 4     | Incomplete response (token limit)      |
| 65900    | gemini-2.5-flash-lite-preview-06-17 | 5     | NoneType object (subscript)            |
| 43915    | gpt-4.1-nano                        | 1     | Incomplete response (token limit)      |
| 43915    | claude-3-5-sonnet-20241022          | 5     | Incomplete response (token limit)      |
| 78467    | claude-sonnet-4-20250514            | 1     | Incomplete response (token limit)      |
| 78467    | claude-3-haiku-20240307             | 1     | Incomplete response (token limit)      |
| 78467    | gemini-2.5-flash                    | 2     | Incomplete response (token limit)      |
| 78467    | gemini-2.5-flash-lite-preview-06-17 | 2     | Incomplete response (token limit)      |
| 78467    | claude-sonnet-4-20250514            | 2     | Incomplete response (token limit)      |
| 78467    | claude-3-haiku-20240307             | 2     | Incomplete response (token limit)      |
| 78467    | claude-sonnet-4-20250514            | 3     | Incomplete response (token limit)      |
| 78467    | claude-3-haiku-20240307             | 3     | Incomplete response (token limit)      |
| 78467    | claude-sonnet-4-20250514            | 4     | Incomplete response (token limit)      |
| 78467    | claude-3-haiku-20240307             | 4     | Incomplete response (token limit)      |
| 78467    | gemini-2.5-flash                    | 5     | Incomplete response (token limit)      |
| 78467    | claude-sonnet-4-20250514            | 5     | Incomplete response (token limit)      |
| 76539    | claude-3-haiku-20240307             | 1     | Incomplete response (token limit)      |
| 76539    | claude-3-haiku-20240307             | 2     | Incomplete response (token limit)      |
| 76539    | claude-3-haiku-20240307             | 3     | Incomplete response (token limit)      |
| 76539    | claude-3-haiku-20240307             | 4     | Incomplete response (token limit)      |
| 76539    | claude-3-haiku-20240307             | 5     | Incomplete response (token limit)      |
| 76550    | o3-mini                             | 1     | Incomplete response (token limit)      |

In [ ]:
# Read the evaluation results from a specific file
with open('auto_evaluation_results\evaluation_results_json\ESSAY_43915_evaluation_results_trial_3.json', 'r', encoding='utf-8') as f:
    # Read lines after "Model: gpt-4.1-nano" and before "Model: gpt-4o"
    for i, line in enumerate(f):
        if 'Model: gpt-4.1-nano' in line:
            start_index = i + 1
        elif 'Model: gpt-4o' in line:
            end_index = i
            break
    evaluation_results = f.readlines()[start_index:end_index]

    for line in f:
        print(line.strip())

In [ ]:
scores_df

In [ ]:
scores_df.to_csv('essay_evaluation_scores.csv', index=False, encoding='utf-8-sig')

In [ ]:
models_to_remove = set()

In [ ]:
# Count how many times each model was used
model_counts = scores_df['model'].value_counts().reset_index()
model_counts.columns = ['model', 'count']

# Identify models that were used 25 times or less
models_to_remove.update(model_counts[model_counts['count'] <= 25]['model'].tolist())

In [ ]:
scores_std_df = scores_df.groupby(['essay_id', 'model'])[['문법의 정확성', '단어 사용의 적절성','문장 표현의 적절성', '문단 내 구조의 적절성', '문단 간 구조의 적절성', '구조의 일관성', '분량의 적절성',
    '주제의 명료성', '설명의 구체성', '사고의 창의성', '프롬프트 독해']].std()

scores_avg_std_df = scores_std_df.groupby('model')[['문법의 정확성', '단어 사용의 적절성','문장 표현의 적절성', '문단 내 구조의 적절성', '문단 간 구조의 적절성', '구조의 일관성', '분량의 적절성',
    '주제의 명료성', '설명의 구체성', '사고의 창의성', '프롬프트 독해']].mean()
scores_avg_std_df['mean_std'] = scores_avg_std_df.mean(axis=1, numeric_only=True)

In [ ]:
models_to_remove.update(scores_avg_std_df[scores_avg_std_df['mean_std'] >= 0.1].index.tolist())

In [ ]:
models_to_remove

In [ ]:
set(models_list) - models_to_remove

In [ ]:
gpt_4_1_scores = scores_df[(scores_df['model'] == 'gpt-4.1')].sort_values(by=['essay_id', 'trial']).reset_index(drop=True)
gpt_4_1_scores

In [ ]:
gemini_2_5_flash_lite_scores = scores_df[(scores_df['model'] == 'gemini-2.5-flash-lite-preview-06-17')].sort_values(by=['essay_id', 'trial']).reset_index(drop=True)
gemini_2_5_flash_lite_scores